# Summer 2022 Data Science Intern Challenge 

By: Tristan Ro

In [11]:
import pandas as pd

#import csv
sneakerSales = pd.read_csv(r'2022_Data_Set.csv')

#### Data Exploration

In [12]:
#find general stats associated with AOV
sneakerSales['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

As seen above, the standard deviation and maximum value of AOV is quite large, which is indicates the presence of outliers.
We can analyze the data further by sorting by order_amount.

In [13]:
sneakerSales.sort_values(by=['order_amount'], inplace=True, ascending=False)
sneakerSales.head(75)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
3332,3333,42,607,704000,2000,credit_card,2017-03-24 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
...,...,...,...,...,...,...,...
938,939,42,808,1056,3,credit_card,2017-03-13 23:43:45
3077,3078,89,754,980,5,debit,2017-03-13 5:27:58
2494,2495,50,757,965,5,debit,2017-03-04 7:32:45
1563,1564,91,934,960,6,debit,2017-03-23 8:25:49


As shown above, shop 42 seems to be the anomaly. We can analyse the unit price for each vendor to see if this correlates. 
Since each shop only sells one model of shoe, we can divide the order amount by the total items.

In [16]:
#create new column with shop's unit price
sneakerSales['unit_price'] = sneakerSales['order_amount']/sneakerSales['total_items']

#create new df for visualization
shop_unit_prices = sneakerSales[['shop_id', 'unit_price']].copy()
shop_unit_prices.sort_values(by=['unit_price'], inplace=True, ascending=False)
shop_unit_prices = shop_unit_prices.drop_duplicates()
shop_unit_prices['unit_price']

40      352.0
1917    201.0
3102    196.0
3315    195.0
558     193.0
        ...  
615     112.0
846     111.0
3387    101.0
1276     94.0
3363     90.0
Name: unit_price, Length: 99, dtype: float64

Based on the initial evaluation of the data, There were two shops in particular that were outliers. Shop 42 seems to sell wholesale (large quantities), while Shop 78 sells high-end sneakers (high unit-price).

We can adjust our AOV by removing outliers to form a more comprehensive value. 
Generally speaking, any value outside the range of (Q1 - 1.5*IQR and Q3 + 1.5*IQR) is deemed an outlier

In [15]:
#calculate IQR
Q1 = sneakerSales['order_amount'].quantile(0.25)
Q3 = sneakerSales['order_amount'].quantile(0.75)
IQR = Q3 - Q1

#eliminate outliers
sneakerSales.drop(sneakerSales.index[sneakerSales.order_amount > Q3 + 1.5*IQR], inplace = True)
sneakerSales.drop(sneakerSales.index[sneakerSales.order_amount < Q1 - 1.5*IQR], inplace = True)
sneakerSales.order_amount.describe()



count    4859.000000
mean      293.715374
std       144.453395
min        90.000000
25%       162.000000
50%       280.000000
75%       380.000000
max       730.000000
Name: order_amount, dtype: float64

These statistics look much better and so the updated average order value is $290.76